<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [1]:
import numpy as np
from lxml import html, etree
import requests
import random

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint, LambdaCallback
from tensorflow.keras.utils import get_file

In [2]:
# scrapes webpage and creates html tree with lxml
page = requests.get('https://www.gutenberg.org/files/100/100-0.txt')
tree = html.fromstring(page.content)

# sets all text to a variable
text_big = tree.text

In [3]:
# scrapes shorter shakespear collection for testing
page_small = requests.get('https://cs.stanford.edu/people/karpathy/char-rnn/shakespear.txt')
tree_small = html.fromstring(page_small.content)

# sets all text to a variable
text_small = tree_small.text

In [4]:
# picks text to use in model
text = text_big

# creates orderedlist of charecters in text
chars = ' '.join(text)

# gets list of unique charecters
unique_chars = list(set(chars))

# creates a dictionary to map number to charecter
num_to_char = dict(enumerate(unique_chars))

# creates dictonary maping char to num
char_to_num = {value: key for key, value in num_to_char.items()}

# encodes text
text_encoded = [char_to_num[i] for i in text_small]

In [5]:
# hyperparameters
iters = 3
sequence_length = 40 # how far back to look
batch_size = round((len(text_encoded) / sequence_length) + 0.5) # gets batch size based on sequence length
hidden_size = 100  # number of hidden nodes
learning_rate = 0.01
num_chars = len(unique_chars)


# model parameters
W_xh = np.random.randn(hidden_size, num_chars) * 0.01     # input to hidden weight
W_hh = np.random.randn(hidden_size, hidden_size) * 0.01   # hidden to hidden weight
W_hy = np.random.randn(num_chars, hidden_size) * 0.01     # hidden to output weight

b_h = np.zeros((hidden_size, 1)) # hidden bias
b_y = np.zeros((num_chars, 1)) # output bias

h_prev = np.zeros((hidden_size, 1)) # not sure

In [6]:
# takes forward/backprop from lecture

def forwardprop(inputs, targets, h_prev):
        
    xs, hs, ys, ps = {}, {}, {}, {} # sets values equal to empty dict
    hs[-1] = np.copy(h_prev) # Copy previous hidden state vector to -1 key value.
    loss = 0 # loss initialization
    
    for t in range(len(inputs)): # t is a "time step" and is used as a key(dic).  
        
        xs[t] = np.zeros((num_chars,1)) 
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(W_xh, xs[t]) + np.dot(W_hh, hs[t-1]) + b_h) # hidden state. 
        ys[t] = np.dot(W_hy, hs[t]) + b_y # unnormalized log probabilities for next chars
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars. 
        
        # Softmax. -> The sum of probabilities is 1 even without the exp() function, but all of the elements are positive through the exp() function.
        loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss). Efficient and simple code     

    return loss, ps, hs, xs


def backprop(ps, inputs, hs, xs, targets):

    dWxh, dWhh, dWhy = np.zeros_like(W_xh), np.zeros_like(W_hh), np.zeros_like(W_hy) # make all zero matrices.
    dbh, dby = np.zeros_like(b_h), np.zeros_like(b_y)
    dhnext = np.zeros_like(hs[0]) # (hidden_size,1) 

    # reversed
    for t in reversed(range(len(inputs))):
        dy = np.copy(ps[t]) # shape (num_chars,1).  "dy" means "dloss/dy"
        dy[targets[t]] -= 1 # backprop into y. After taking the soft max in the input vector, subtract 1 from the value of the element corresponding to the correct label.
        dWhy += np.dot(dy, hs[t].T)
        dby += dy 
        dh = np.dot(W_hy.T, dy) + dhnext # backprop into h. 
        dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity #tanh'(x) = 1-tanh^2(x)
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        dhnext = np.dot(W_hh.T, dhraw)
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]: 
        np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients.  
    
    return dWxh, dWhh, dWhy, dbh, dby

In [7]:
%%time

data_pointer = 0

# memory variables for Adagrad
mWxh, mWhh, mWhy = np.zeros_like(W_xh), np.zeros_like(W_hh), np.zeros_like(W_hy)
mbh, mby = np.zeros_like(b_h), np.zeros_like(b_y) 

for i in range(iters):
    h_prev = np.zeros((hidden_size, 1)) # reset RNN memory
    data_pointer = 0 # go from start of data
    
    for b in range(batch_size):
        
        inputs = [char_to_num[ch] for ch in chars[data_pointer: data_pointer + sequence_length]]
        targets = [char_to_num[ch] for ch in chars[data_pointer + 1: data_pointer + sequence_length + 1]] # t+1        
            
        if (data_pointer + sequence_length + 1 >= len(chars) and b == batch_size - 1): # processing of the last part of the input data. 
            targets.append(char_to_int[" "])   # When the data doesn't fit, add space(" ") to the back.

        # forward prop
        loss, ps, hs, xs = forwardprop(inputs, targets, h_prev)

        # backward prop
        dWxh, dWhh, dWhy, dbh, dby = backprop(ps, inputs, hs, xs, targets) 
        
        
    # perform parameter update with Adagrad
        for param, dparam, mem in zip([W_xh, W_hh, W_hy, b_h, b_y], 
                                    [dWxh, dWhh, dWhy, dbh, dby], 
                                    [mWxh, mWhh, mWhy, mbh, mby]):
            mem += dparam * dparam # elementwise
            param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update      
    
        data_pointer += sequence_length # move data pointer
        
    if i % 1 == 0:
        print ('iter %d, loss: %f' % (i, loss)) # print progress

iter 0, loss: 56.560689
iter 1, loss: 55.672149
iter 2, loss: 52.398502
CPU times: user 1min 37s, sys: 5min 50s, total: 7min 27s
Wall time: 1min 2s


In [8]:
# checking the value of input and target

data_pointer=0

test = [char_to_num[ch] for ch in chars[data_pointer+10: data_pointer + sequence_length +10]]

words = []
for i in test:
    word = num_to_char[i]
    words.append(word)
    
string = ''
for i in words:
    string += i
    
print(string)

c t   G u t e n b e r g ’ s   T h e   C 


In [9]:
def predict(test_char, length):
    x = np.zeros((num_chars, 1)) 
    x[char_to_num[test_char]] = 1
    ixes = []
    h = np.zeros((hidden_size,1))

    for t in range(length):
        h = np.tanh(np.dot(W_xh, x) + np.dot(W_hh, h) + b_h) 
        y = np.dot(W_hy, h) + b_y
        p = np.exp(y) / np.sum(np.exp(y)) 
        ix = np.random.choice(range(num_chars), p=p.ravel()) # ravel -> rank0
        # "ix" is a list of indexes selected according to the soft max probability.
        x = np.zeros((num_chars, 1)) # init
        x[ix] = 1 
        ixes.append(ix) # list
    txt = test_char + ''.join(num_to_char[i] for i in ixes)
    print ('----\n %s \n----' % (txt, ))

In [10]:
predict('a', 50)

----
 a n h S n f   a c e g p   e e   
 i p o 
 o s a 
----


# Using Keras

In [11]:
# picks text to use in model
text = text_small
# creates orderedlist of charecters in text
chars = ' '.join(text_small)

# gets list of unique charecters
unique_chars = list(set(chars))

# creates a dictionary to map number to charecter
num_to_char = dict(enumerate(unique_chars))

# creates dictonary maping char to num
char_to_num = {value: key for key, value in num_to_char.items()}

# encodes text
text_encoded = [char_to_num[i] for i in text_small]

In [12]:
maxlen = 40
step = 3
sentences = []
next_chars = []


for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 33318


In [13]:
# looks at sentences
sentences[:5]

["That, poor contempt, or claim'd thou sle",
 "t, poor contempt, or claim'd thou slept ",
 "poor contempt, or claim'd thou slept so ",
 "r contempt, or claim'd thou slept so fai",
 "ontempt, or claim'd thou slept so faithf"]

In [14]:
# looks at next chars
next_chars[:5]

['p', 's', 'f', 't', 'u']

In [15]:
# vectorize
x = np.zeros((len(sentences), maxlen, len(unique_chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(unique_chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_to_num[char]] = 1
    y[i, char_to_num[next_chars[i]]] = 1

In [16]:
x[0].shape

(40, 62)

In [17]:
print(x[0][0].shape)
x[0][0]

(62,)


array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False])

In [18]:
y[0] # just realized we are using last 40 words (x) as input to predict next letter (y)

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False])

In [19]:
# build model
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(unique_chars))))
model.add(Dense(len(unique_chars), activation='softmax'))

optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [20]:
# fit model and set output to history var
history = model.fit(x, 
                    y,
                    batch_size=128,
                    epochs=10)

Train on 33318 samples
Epoch 1/10
33318/33318 [==============================] - 5s 164us/sample - loss: 2.5773
Epoch 2/10
33318/33318 [==============================] - 2s 74us/sample - loss: 2.0644
Epoch 3/10
33318/33318 [==============================] - 2s 72us/sample - loss: 1.8842
Epoch 4/10
33318/33318 [==============================] - 2s 73us/sample - loss: 1.7514
Epoch 5/10
33318/33318 [==============================] - 2s 73us/sample - loss: 1.6428
Epoch 6/10
33318/33318 [==============================] - 2s 73us/sample - loss: 1.5451
Epoch 7/10
33318/33318 [==============================] - 2s 73us/sample - loss: 1.4587
Epoch 8/10
33318/33318 [==============================] - 2s 74us/sample - loss: 1.3908
Epoch 9/10
33318/33318 [==============================] - 2s 73us/sample - loss: 1.3205
Epoch 10/10
33318/33318 [==============================] - 2s 74us/sample - loss: 1.2557


In [21]:
start_index = random.randint(0, len(text) - maxlen - 1)

In [22]:
sentence = text[start_index: start_index + maxlen]
sentence

' sweet good stocks, and marry with her w'

In [23]:
x_pred = np.zeros((1, maxlen, len(unique_chars)))
for t, i in enumerate(sentence):
    x_pred[0, t, char_to_num[i]] = 1


In [24]:
x_pred.shape

(1, 40, 62)

In [27]:
x_pred

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

In [29]:
sm_prediction = model.predict(x_pred)

In [32]:
# this is the probabilities for each charecter being the next char given x pred as the start
sm_prediction[0].sum()

1.0

In [42]:
best = np.amax(sm_prediction)

In [46]:
sm_prediction/best

array([[3.64090338e-05, 2.11246149e-08, 3.02720146e-05, 8.64725189e-06,
        1.22183735e-07, 3.47687751e-06, 2.44201624e-06, 1.47042971e-03,
        6.42986242e-06, 5.63208487e-05, 2.20692164e-05, 5.37290752e-01,
        4.49101748e-07, 3.75856034e-04, 1.80732549e-08, 4.33581328e-04,
        6.69973160e-05, 2.66016570e-10, 2.66223597e-05, 4.39468586e-08,
        1.20934684e-08, 5.80301332e-07, 1.47466994e-06, 4.26553470e-07,
        1.69343313e-08, 3.88917059e-08, 4.06579602e-06, 2.22693441e-09,
        1.54655045e-05, 3.26412510e-06, 1.28146223e-06, 2.60664383e-03,
        5.12294901e-05, 3.83335390e-08, 7.40756789e-08, 4.47359198e-04,
        1.52123945e-07, 2.96765083e-05, 1.29370292e-05, 6.91521564e-06,
        1.92814216e-04, 1.90886418e-09, 3.97080324e-09, 4.52901393e-01,
        4.21837103e-06, 9.61817932e-07, 8.10102210e-06, 3.60662307e-06,
        1.00000000e+00, 1.23759073e-05, 1.31652132e-01, 8.24770936e-08,
        5.42856753e-07, 1.21815000e-02, 4.03077956e-06, 1.335941

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN